In [12]:
import sys
import os


from AutoEncoderModule import *

import torch
import torch.nn as nn
import pickle
import pandas as pd
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torchsummary import summary
from torchmetrics.regression import R2Score, MeanAbsoluteError, MeanAbsolutePercentageError, MeanSquaredError
import matplotlib.pyplot as plt

In [13]:
DATA_PATH = '../Data/'

electric_df = pd.read_csv(DATA_PATH + 'electric_df_clear_2.csv', index_col = 0)
water_df = pd.read_csv(DATA_PATH + 'water_diff_df_clear.csv')

In [14]:
print(electric_df.head())
print()
print(water_df.head())

               0     1     2     3     4     5     6     7     8     9  ...  \
Unnamed: 0                                                              ...   
0           1.23  1.45  1.73  1.62  1.31  1.48  1.74  1.71  1.30  1.49  ...   
1           1.45  1.73  1.62  1.31  1.48  1.74  1.71  1.30  1.49  1.94  ...   
2           1.73  1.62  1.31  1.48  1.74  1.71  1.30  1.49  1.94  1.68  ...   
3           1.62  1.31  1.48  1.74  1.71  1.30  1.49  1.94  1.68  1.26  ...   
4           1.31  1.48  1.74  1.71  1.30  1.49  1.94  1.68  1.26  1.39  ...   

              18    19    20    21    22    23    24    25    26    27  
Unnamed: 0                                                              
0           1.77  1.68  1.20  1.42  1.88  1.67  1.24  1.39  1.67  1.60  
1           1.68  1.20  1.42  1.88  1.67  1.24  1.39  1.67  1.60  1.26  
2           1.20  1.42  1.88  1.67  1.24  1.39  1.67  1.60  1.26  1.41  
3           1.42  1.88  1.67  1.24  1.39  1.67  1.60  1.26  1.41  1.68  
4       

In [15]:
# electric_features = electric_df[electric_df.columns[:]]
# electric_target = electric_df[electric_df.columns[-1:]]

# electric_X_train, electric_X_test, electric_y_train, electric_y_test = train_test_split(electric_features,
#                                                     electric_target,
#                                                     random_state = 42,
#                                                     test_size = 0.2)

# water_features = water_df[water_df.columns[:]]
# water_target = water_df[water_df.columns[-1:]]

# water_X_train, water_X_test, water_y_train, water_y_test = train_test_split(water_features,
#                                                                             water_target,
#                                                                             random_state = 42,
#                                                                             test_size = 0.2)

In [16]:
EPOCH = 10000
BATCH_SIZE = 64
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

In [17]:
electric_trainDS = CustomDataset(electric_df)
water_trainDS = CustomDataset(water_df)

electric_trainDL = DataLoader(electric_trainDS, batch_size = BATCH_SIZE)
water_trainDL = DataLoader(water_trainDS, batch_size = BATCH_SIZE)

In [18]:
input_size = 28
hidden_dim = 256
penalty = 0.5
threshold = 0.02


vanilla_model = VanillaModel(input_size = input_size, hidden_dim = hidden_dim)

optimizer = optim.Adam(vanilla_model.parameters(), lr = LR)

scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min', patience = 10, verbose = True)


c:\Users\KDP-2\anaconda3\envs\Project_38\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [19]:
summary(vanilla_model, input_size = (28, ))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 256]           7,424
              ReLU-2                  [-1, 256]               0
            Linear-3                  [-1, 128]          32,896
              ReLU-4                  [-1, 128]               0
            Linear-5                   [-1, 64]           8,256
              ReLU-6                   [-1, 64]               0
            Linear-7                   [-1, 32]           2,080
              ReLU-8                   [-1, 32]               0
            Linear-9                   [-1, 16]             528
             ReLU-10                   [-1, 16]               0
           Linear-11                    [-1, 8]             136
             ReLU-12                    [-1, 8]               0
           Linear-13                   [-1, 16]             144
             ReLU-14                   

In [20]:
for a, b in water_trainDL:
    print(len(a), len(b))
    break

64 64


In [ ]:
vae_loss = training(vanilla_model, electric_trainDL,
                    optimizer, penalty, threshold, EPOCH, scheduler, DEVICE)

[1 / 10000]
- TRAIN LOSS : 130.18670622825314
tensor([1.4200, 1.4800, 1.6400, 1.5300, 1.4600, 1.4400, 1.6300, 1.5400, 1.4600,
        1.4700, 1.5900, 1.3700, 1.3300, 1.3600, 1.4500, 1.6900, 1.7900, 1.4400,
        1.6400, 1.5100, 1.4600, 1.3600, 1.4500, 1.6900, 1.7900, 1.4400, 1.6400,
        1.5100])
tensor([1.4952, 1.4705, 1.5312, 1.5656, 1.4941, 1.4614, 1.5355, 1.5840, 1.5072,
        1.4668, 1.5425, 1.5881, 1.5061, 1.4625, 1.5437, 1.5943, 1.5055, 1.4751,
        1.5489, 1.5849, 1.4996, 1.4673, 1.5338, 1.5774, 1.5151, 1.4864, 1.5443,
        1.5733], grad_fn=<SelectBackward0>)
[2 / 10000]
- TRAIN LOSS : 8.665096514757499
tensor([1.4200, 1.4800, 1.6400, 1.5300, 1.4600, 1.4400, 1.6300, 1.5400, 1.4600,
        1.4700, 1.5900, 1.3700, 1.3300, 1.3600, 1.4500, 1.6900, 1.7900, 1.4400,
        1.6400, 1.5100, 1.4600, 1.3600, 1.4500, 1.6900, 1.7900, 1.4400, 1.6400,
        1.5100])
tensor([1.4591, 1.4257, 1.4950, 1.5436, 1.4603, 1.4198, 1.4980, 1.5596, 1.4678,
        1.4180, 1.4999, 1.5656,

In [ ]:
plt.figure(figsize = (12, 8))
plt.plot(vae_loss)
plt.show()

In [ ]:
for i in range(0, 31, 15):
    print(i)

0
15
30
